In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from tqdm import tqdm
import numpy as np


python_path = "/home/yskim/projects/sparse-bfp"
import sys
if python_path not in sys.path:
    sys.path.append(python_path)

from util.reprod_util import set_reproducibility

from util.custom_linear import CustomLinear, PrecisionFlag
from util.custom_conv2d import CustomConv2d

set_reproducibility(random_seed=128)

from util.data_load import *

from util.bfp.bfp_config import BfpConfig
BfpConfig.bfp_M_Bit = 8

In [ ]:
batch_size = 32
# train_loader, valid_loader = generate_data_loaders(type=DL_TYPE_IMAGENET, batch_size=32)
train_loader, valid_loader = generate_data_loaders(type=DL_TYPE_CIFAR_100, batch_size=batch_size)

In [ ]:
def wrap_modules(module, precision_flag: PrecisionFlag):
    for name, _ in module.named_children():
        child = getattr(module, name)
        child_cnt = 0
        for _ in child.children():
            child_cnt += 1

        if child_cnt == 0:
            layer = getattr(module, name)
            for _, param in layer.named_parameters():
                if param.requires_grad:
                    pass
                    # print(param.shape)
            # print()
            if "Linear" in str(layer):
                custom_layer = CustomLinear(input_features=layer.in_features, 
                                            output_features=layer.out_features, 
                                            bias=True if layer.bias is not None else False, 
                                            precision_flag=precision_flag)
                custom_layer.weight = nn.Parameter(layer.weight.data.clone().detach())
                if layer.bias is not None:
                    custom_layer.bias = nn.Parameter(layer.bias.data.clone().detach())
                setattr(module, name, custom_layer)
            elif "Conv" in str(layer):
                custom_layer = CustomConv2d(in_channels=layer.in_channels,
                                            out_channels=layer.out_channels,
                                            kernel_size=layer.kernel_size[0],
                                            stride=layer.stride[0],
                                            padding=layer.padding[0],
                                            bias=True if layer.bias is not None else False,
                                            precision_flag=precision_flag)
                custom_layer.weight = nn.Parameter(layer.weight.data.clone().detach())
                if layer.bias is not None:
                    custom_layer.bias = nn.Parameter(layer.bias.data.clone().detach())
                setattr(module, name, custom_layer)
        else:
            wrap_modules(child, precision_flag)


# def change_to_leaky_relu(module):
#     for name, _ in module.named_children():
#         child = getattr(module, name)
#         child_cnt = 0
#         for _ in child.children():
#             child_cnt += 1

#         if child_cnt == 0:
#             layer = getattr(module, name)
#             for _, param in layer.named_parameters():
#                 if param.requires_grad:
#                     pass
#                     # print(param.shape)
#             # print()
#             if "ReLU" in str(layer):
#                 leaky_relu = nn.LeakyReLU()
#                 setattr(module, name, leaky_relu)
#                 print("changed!")
#         else:
#             change_to_leaky_relu(child)

In [ ]:
# model_ref = CustomAlexNetCIFAROri(num_classes=100).to('cuda')
# model_ref = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet20", pretrained=False).to('cuda')
# model_ref = models.resnet18(pretrained=False).to('cuda')


device = 'cuda'

model_ref = models.mobilenet_v2(num_classes=100)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ref.parameters(), lr=0.001, weight_decay=1e-3)
# optimizer = optim.SGD(model_ref.parameters(), lr=0.01)

# wrap_modules(model_ref, precision_flag=PrecisionFlag.BFP)
# change_to_leaky_relu(model_ref)

model_ref = model_ref.to(device)
criterion = criterion.to(device)

log_root = "/home/yskim/projects/sparse-bfp/train-log"

model_name = "tv-mobilenet_v2"
data_format = "fp"

log_name = f"{model_name}-b{batch_size}-{data_format}-{device}"

path = f"{log_root}/{log_name}"

if not os.path.isdir(path):
    os.mkdir(path)

torch.save({
    'model_state_dict': model_ref.state_dict(),
}, f"{path}/init-weights.pth")

model_ref = model_ref.to(device)
criterion = criterion.to(device)


best_valid_acc = float("-inf")

epochs = 100
for epoch in range(epochs):
    model_ref.train()
    print(f"epch {epoch}th")

    train_top1_correct_cnt = 0
    train_top5_correct_cnt = 0
    train_total_cnt = 0
    train_loss = 0
    train_batch_cnt = 0

    for batch_index, (inputs, targets) in enumerate(tqdm(train_loader), 0):
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        
        train_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                train_top5_correct_cnt += 1
                continue

        train_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)

        train_loss += loss.item() * outputs.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= train_total_cnt
    train_top1_acc = train_top1_correct_cnt / train_total_cnt * 100.0
    train_top5_acc = train_top5_correct_cnt / train_total_cnt * 100.0

    print(f"[train accuracy] "
          f"top1: {train_top1_acc:.2f}%, "
          f"top5: {train_top5_acc:.2f}%")
    print(f"[train loss]: {train_loss}")

    model_ref.eval()

    valid_top1_correct_cnt = 0
    valid_top5_correct_cnt = 0
    valid_total_cnt = 0
    valid_loss = 0
    valid_batch_cnt = 0

    for batch_index, (inputs, targets) in enumerate(tqdm(valid_loader), 0):
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        valid_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                valid_top5_correct_cnt += 1
                continue

        valid_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)
        valid_loss += loss.item() * outputs.shape[0]

    valid_loss /= valid_total_cnt
    valid_top1_acc = valid_top1_correct_cnt / valid_total_cnt * 100.0
    valid_top5_acc = valid_top5_correct_cnt / valid_total_cnt * 100.0


    print(f"[valid accuracy] "
          f"top1: {valid_top1_acc:.2f}%, "
          f"top5: {valid_top5_acc:.2f}%")
    print(f"[valid loss]: {valid_loss}")

    if best_valid_acc < valid_top5_acc:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_ref.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_top1_accuracy': train_top1_acc,
            'train_top5_accuracy': train_top5_acc,
            'valid_loss': valid_loss,
            'valid_top1_accuracy': valid_top1_acc,
            'valid_top5_accuracy': valid_top5_acc,
        }, f"{path}/best-top5-accuracy.pth")

        best_valid_acc = valid_top5_acc

    if epoch % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_ref.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_top1_accuracy': train_top1_acc,
            'train_top5_accuracy': train_top5_acc,
            'valid_loss': valid_loss,
            'valid_top1_accuracy': valid_top1_acc,
            'valid_top5_accuracy': valid_top5_acc,
        }, f"{path}/epoch-{epoch}.pth")